#Hàm

In [ ]:
import os
os.chdir('/content/drive/MyDrive/DA_ChuyenNganhKHDL')

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
%%writefile urlmarker.py

ANY_URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""

WEB_URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""

IP_REGEX = r"""[0-9]+(?:\.[0-9]+){3}:[0-9]+"""


Overwriting urlmarker.py


In [ ]:
%%writefile utils.py
import ast, os

"""
load_n_gram(corpus_path): load data from corpus_path
"""
def load_n_gram(corpus_path):
  with open(corpus_path, encoding = 'utf-8') as file:
    words = file.read()
    words = ast.literal_eval(words)
    return words

"""
load_file_with_newline: load words from '<file_path>.txt' file separate with \n notation
Output: Set of words
Stopwords.txt description: 1 word 1 line + 2 python style commments
"""
def load_file_with_newline(file_path):
  words = set()
  with open(file_path, encoding='utf-8') as file:
    for line in file:
      li=line.strip()
      if not li.startswith("#"):
        words.add(line.rstrip())

  return words


Overwriting utils.py


In [ ]:
%%writefile text_utils.py
import numpy as np
import pandas as pd
import json, os, ast, re, sys
import csv
import unicodedata as ud
import sys
import re
import urlmarker

"""
  token_sylabling: String -> Sylable Token
  input: text - Unicode String
  output: List() of token
"""
def token_sylabling(text):
    text = ud.normalize("NFC", text)

    sign = ["==>", "=>", "->", "\.\.\.", ">>"]
    digits = "\d+([\.,_]\d+)?"
    email = "[\w\.-]+@[\w\.-]+"
    web = urlmarker.WEB_URL_REGEX
    datetime = [
        "\d{1,2}\/\d{1,2}(\/\d+)?",
        "\d{1,2}-\d{1,2}(-\d+)?",
    ]
    word = "\w+"
    non_word = "[^\w\s]"
    abbreviations = [
        "[A-ZĐ]+\.",
        "Tp\.?",
        "Mr\.", "Mrs\.", "Ms\.",
        "Dr\.?", "ThS\.?", "TS\.?", "GS\.?", "PSG\.?"
    ]

    patterns = []
    patterns.extend(abbreviations)
    patterns.extend(sign)
    patterns.extend(datetime)
    patterns.extend([web, email])
    patterns.extend([digits, non_word, word])
    patterns = "(" + "|".join(patterns) + ")"

    if sys.version_info < (3, 0):
            patterns = patterns.decode("utf-8")
    tokens = re.findall(patterns, text, re.UNICODE)
    return [token[0] for token in tokens]

"""
  remove_stopwords: remove "stopwords" from "paragraph"
  input:
    + stopwords: Set() of stopwords
    + paragraph: List() of word in paragraph
  output: List() of words after remove stopwords
"""
def remove_stopwords(paragraph, stopwords):
  new_para = []
  for word in paragraph:
    if not word in stopwords:
      new_para.append(word)
  return new_para

"""
  remove_punc: remove punctuation from text
  input:
    + text: String Type
  output: text after remove all punctuations
"""
def remove_punc(text):
  tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if ud.category(chr(i)).startswith("P"))
  return text.translate(tbl)

"""
  is_word: Check if <string> is a word
"""
def is_word(string):
  sign = ["==>", "=>", "->", "\.\.\.", ">>"]
  digits = "\d+([\.,_]\d+)?"
  email = "[\w\.-]+@[\w\.-]+"
  web = urlmarker.WEB_URL_REGEX
  datetime = [
      "\d{1,2}\/\d{1,2}(\/\d+)?",
      "\d{1,2}-\d{1,2}(-\d+)?",
  ]
  non_word = "[^\w\s]"
  abbreviations = [
      "[A-ZĐ]+\.",
      "Tp\.?",
      "Mr\.", "Mrs\.", "Ms\.",
      "Dr\.?", "ThS\.?", "TS\.?", "GS\.?", "PSG\.?"
  ]

  patterns = []
  patterns.extend(abbreviations)
  patterns.extend(sign)
  patterns.extend(datetime)
  patterns.extend([web, email])
  patterns.extend([digits, non_word])
  patterns = "(" + "|".join(patterns) + ")"
  patterns = re.compile(patterns)
  return not bool(patterns.match(string))

Overwriting text_utils.py


#Xử lý data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import json, os, ast, re, sys
import csv
import unicodedata as ud
import random

from text_utils import remove_stopwords, token_sylabling
from utils import load_file_with_newline
import urlmarker

In [ ]:
path_text = '/content/drive/MyDrive/DA_ChuyenNganhKHDL/Data/vn_news_226_tlfr.csv'
data = pd.read_csv(path_text)
data.head()

,text,label
0,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...,1
1,Thủ tướng Nhật cúi đầu xin lỗi vì tinh thần ph...,1
2,Choáng! Cơ trưởng đeo khăn quàng quẩy banh nóc...,1
3,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!\r...,1
4,"Đại học Hutech sẽ áp dụng cải cách ""Tiếq Việt""...",1


In [ ]:
data.shape

(226, 2)

Đường dẫn đến các file thư viện

In [ ]:
dict_path = '/content/drive/MyDrive/DA_ChuyenNganhKHDL/Dictionaries'

bi_gram_path = os.path.join(dict_path, 'bi_gram.txt')
tri_gram_path = os.path.join(dict_path, 'tri_gram.txt')
four_gram_path = os.path.join(dict_path, 'four_gram.txt')
stopword_path = os.path.join(dict_path, 'Stopwords_vi.txt')

Tải các từ điển đó và đưa vào Set()

In [ ]:
bi_dict = load_file_with_newline(bi_gram_path)
tri_dict = load_file_with_newline(tri_gram_path)
four_dict = load_file_with_newline(four_gram_path)

stopwords = load_file_with_newline(stopword_path)

Đưa các từ trong từ điển về chữ thường

In [ ]:
bi_gram = [x.lower() for x in bi_dict]
tri_gram = [x.lower() for x in tri_dict]
four_gram = [x.lower() for x in four_dict]

Thuật toán ___Longest Matching___ gộp các syllable token - tiếng - thành các từ được định nghĩa trong bộ từ điển. Input:
1. ```token```: List của các syllable sau khi tách ra
2. ```bi_gram, tri_gram, four_gram```: Các set() từ điển đã load ở trên

In [ ]:
def LongestMatching(token, bi_gram, tri_gram, four_gram):
    #token_len: Length of syllable list
    token_len = len(token)
    cur_id = 0
    word_list = []

    #done: True when cur_id reach
    done = False

    while (cur_id < token_len) and (not done):
        cur_word = token[cur_id]
        if(cur_id >= token_len - 1):
            word_list.append(cur_word)
            done = True
        else:
            next_word = token[cur_id + 1]
            bi_word = " ".join([cur_word.lower(), next_word.lower()])
            if(cur_id >= token_len - 2):
                if bi_word in bi_gram:
                    word_list.append("_".join([cur_word, next_word]))
                    cur_id  = cur_id + 2
                else:
                    word_list.append(cur_word)
                    cur_id  = cur_id + 1

            else:
                bi_next_word = token[cur_id + 2]
                tri_word = " ".join([bi_word, bi_next_word.lower()])
                if(cur_id >= token_len - 3):
                    if tri_word in tri_gram:
                        word_list.append("_".join([cur_word, next_word, bi_next_word]))
                        cur_id  = cur_id + 3
                    elif bi_word in bi_gram:
                        word_list.append("_".join([cur_word, next_word]))
                        cur_id = cur_id + 2
                    else:
                        word_list.append(cur_word)
                        cur_id = cur_id + 1
                else:
                    tri_next_word = token[cur_id + 3]
                    four_word = " ".join([tri_word, tri_next_word.lower()])
                    if four_word in four_gram:
                        word_list.append("_".join([cur_word, next_word, bi_next_word, tri_next_word]))
                        cur_id  = cur_id + 4
                    elif tri_word in tri_gram:
                        word_list.append("_".join([cur_word, next_word, bi_next_word]))
                        cur_id  = cur_id + 3
                    elif bi_word in bi_gram:
                        word_list.append("_".join([cur_word, next_word]))
                        cur_id = cur_id + 2
                    else:
                        word_list.append(cur_word)
                        cur_id = cur_id + 1
    return word_list

Input: văn bản tin tức => Output: Văn bản của các cụm từ đã được gom cụm theo bộ từ điển:
1. Chuyển đổi text thành các âm tiết bằng hàm: _token_\__sylabling_
2. Dùng longest matching để chuyển các tiếng thành các từ dựa theo bộ từ điển
3. Loại bỏ stopword

In [ ]:
def text_preprocessing(text):
    token_list = token_sylabling(text)
    word_list = LongestMatching(token_list, bi_gram, tri_gram, four_gram)
    remove_stopword_list = remove_stopwords(word_list, stopwords)
    new_text = ' '.join(remove_stopword_list)
    return new_text

In [ ]:
newslist = []

with open(path_text, newline='', encoding = 'UTF-8') as csv_file:
    reader = csv.DictReader(csv_file)
    newslist = list(reader)

In [ ]:
newslist[0]

{'text': 'Thủ tướng Abe cúi đầu xin lỗi vì hành động phi thể thao của tuyển Nhật\r\nTheo Sankei Sports, sáng nay Thủ tướng Nhật Bản Shinzo Abe công khai gửi lời xin lỗi tới Nhật hoàng và toàn bộ người dân vì tinh thần thi đấu phi thể thao của đội tuyển Nhật tại World Cup 2018.\r\n\r\nTối qua, sau lượt trận cuối vòng bảng World Cup 2018, Nhật Bản có cùng chỉ số phụ như Senegal, đội bị loại sau khi thua Colombia. Nhưng Nhật Bản vào vòng sau nhờ chỉ số fair-play vì nhận ít thẻ phạt hơn.\r\nThủ tướng Nhật Bản Shinzo Abe cúi đầu xin lỗi.\r\n\r\n“Với tinh thần của những võ sĩ đạo Samurai, nhưng đội tuyển Nhật Bản đã có những hành động thiếu tinh thần thượng võ trong thi đấu tại World Cup, để lại nhiều chỉ trích và bất bình cho toàn dân, làm mất hình ảnh kiên cường của người dân Nhật Bản trên đấu trường quốc tế. Là người đứng đầu, tôi xin thành thật nhận trách nhiệm và gửi lời xin lỗi sâu sắc tới nhân dân.”, ông Abe cúi đầu nhận trách nhiệm về mình.\r\nNgười Nhật Bản nổi tiếng về tinh thần, t

Xuất kết quả tiền xử lý

In [ ]:
preprocessed_news = []
for news in newslist:
    pre_news = dict()
    pre_news['text'] = text_preprocessing(news['text'])
    pre_news['label'] = news['label']
    preprocessed_news.append(pre_news)

In [ ]:
preprocessed_news[0]

{'text': 'Thủ_tướng Abe cúi đầu xin_lỗi hành_động phi thể_thao tuyển Nhật Theo Sankei Sports , sáng nay Thủ_tướng Nhật_Bản Shinzo Abe công_khai gửi_lời xin_lỗi tới Nhật hoàng toàn_bộ người dân tinh_thần thi_đấu phi thể_thao đội_tuyển Nhật World_Cup 2018 . Tối , lượt trận cuối vòng bảng World_Cup 2018 , Nhật_Bản chỉ_số phụ Senegal , đội loại thua Colombia. Nhưng Nhật_Bản vòng nhờ chỉ_số fair - play nhận ít thẻ phạt hơn . Thủ_tướng Nhật_Bản Shinzo Abe cúi đầu xin_lỗi . “ Với tinh_thần võ_sĩ_đạo Samurai , đội_tuyển Nhật_Bản hành_động thiếu tinh_thần thượng_võ trong thi_đấu World_Cup , để_lại chỉ_trích bất_bình toàn_dân , làm mất hình_ảnh kiên_cường người dân Nhật_Bản đấu_trường quốc_tế . Là người đứng_đầu , tôi xin thành_thật nhận trách_nhiệm gửi_lời xin_lỗi sâu_sắc tới nhân_dân . ” , ông Abe cúi đầu nhận trách_nhiệm về mình . Người Nhật_Bản nổi_tiếng về tinh_thần , trách_nhiệm chất võ_sĩ_đạo . Đó lý_do đội_tuyển áo_xanh mệnh_danh “ Samurai xanh ” . Tuy_nhiên , Nhật_Bản sau_đó nhận chỉ_tr

Lưu dữ liệu

In [ ]:
keys = preprocessed_news[0].keys()
with open('/content/drive/MyDrive/DA_ChuyenNganhKHDL/Data/preproc_vn_news_226_tdlfr.csv', 'w') as csv_file:
    dict_writer = csv.DictWriter(csv_file, fieldnames = keys)
    dict_writer.writeheader()
    for news in preprocessed_news:
        dict_writer.writerow(news)

Thử nghiệm

In [ ]:
text = "Em rất thích ngắm nhìn những cơn mưa mùa xuân. Mưa xuân không vội vã, ồn ào như mưa hạ. Mưa xuân dịu dàng, chầm chậm, mang đến cảm giác bình yên và thư thái trong tâm hồn. Những hạt mưa bắt đầu rơi, hạt mưa nhỏ, lất phất trong gió, vương những giọt long lanh trên cánh đào mỏng manh, trên những chồi non cây lá."
token_list = token_sylabling(text)
print(token_list)

['Em', 'rất', 'thích', 'ngắm', 'nhìn', 'những', 'cơn', 'mưa', 'mùa', 'xuân', '.', 'Mưa', 'xuân', 'không', 'vội', 'vã', ',', 'ồn', 'ào', 'như', 'mưa', 'hạ', '.', 'Mưa', 'xuân', 'dịu', 'dàng', ',', 'chầm', 'chậm', ',', 'mang', 'đến', 'cảm', 'giác', 'bình', 'yên', 'và', 'thư', 'thái', 'trong', 'tâm', 'hồn', '.', 'Những', 'hạt', 'mưa', 'bắt', 'đầu', 'rơi', ',', 'hạt', 'mưa', 'nhỏ', ',', 'lất', 'phất', 'trong', 'gió', ',', 'vương', 'những', 'giọt', 'long', 'lanh', 'trên', 'cánh', 'đào', 'mỏng', 'manh', ',', 'trên', 'những', 'chồi', 'non', 'cây', 'lá', '.']


In [ ]:
token_list_1 = LongestMatching(token_list, bi_gram, tri_gram, four_gram)
print(token_list_1)

['Em', 'rất', 'thích', 'ngắm', 'nhìn', 'những', 'cơn_mưa', 'mùa_xuân', '.', 'Mưa', 'xuân', 'không', 'vội_vã', ',', 'ồn_ào', 'như', 'mưa', 'hạ', '.', 'Mưa', 'xuân', 'dịu_dàng', ',', 'chầm_chậm', ',', 'mang', 'đến', 'cảm_giác', 'bình_yên', 'và', 'thư_thái', 'trong', 'tâm_hồn', '.', 'Những', 'hạt_mưa', 'bắt_đầu', 'rơi', ',', 'hạt_mưa', 'nhỏ', ',', 'lất_phất', 'trong', 'gió', ',', 'vương', 'những', 'giọt', 'long_lanh', 'trên', 'cánh', 'đào', 'mỏng_manh', ',', 'trên', 'những', 'chồi', 'non', 'cây', 'lá', '.']


In [ ]:
preprocessing_text = text_preprocessing(text)
print(preprocessing_text)

Em thích ngắm nhìn cơn_mưa mùa_xuân . Mưa xuân vội_vã , ồn_ào mưa hạ . Mưa xuân dịu_dàng , chầm_chậm , mang đến cảm_giác bình_yên thư_thái trong tâm_hồn . Những hạt_mưa bắt_đầu rơi , hạt_mưa nhỏ , lất_phất trong gió , vương giọt long_lanh cánh đào mỏng_manh , chồi non cây lá .
